## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### 1. Открываем файл с данными и изучаем общую информацию. 

In [ ]:
import pandas as pd
data = pd.read_csv('datasets/bank_creditscoring.csv')
data

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В нашей таблице достаточно много корректной информации для решения нашей задачи. На мой взгляд, чтобы ответить на вопрос проекта, важна следующая информация:

- имел ли задолженность по возврату кредитов (`debt`),
- количество детей (`children`),
- семейное положении (`family_status`),
- половая принадлежность (`gender`),
- возраст клиента (`dob_years`),
- цель кредита (`purpose`).


### 2. Предобработка данных

Рассмотрим подробнее каждую и приведём в надлежащий вид.


**Определяем и заполняем пропущенные значения**

В информационной таблице `info()` видно, что в столбцах `days_employed` и `total_income` есть non-null объекты. Воспользуемся методом `.fillna()` для замены отсутствующих данных на конкретные числовые значения. Так как количество данных пропусков достаточно незначительно, можем основываться на средних имеющихся в таблице данных. Причем, замену будем производить на среднюю величину для каждого вида деятельности кредитора `income_type`. Сгруппируем по типу занятости `income_type` и заменим NaN  на соответствующую среднюю величину. В результате, ячейки с пропусками в столбце `days_employed` будут заполнены средней продолжительностью стажа по каждой категории типа занятости заёмщика. А пропущенные ячейки в колонке `total_income` будут заполнены среднеми значениями доходов по данному типу занятости кредитора. 

In [3]:
data['days_employed'] = data.groupby(['income_type'])['days_employed'].transform(lambda x: x.fillna(x.mean()))
data['total_income'] = data.groupby(['income_type'])['total_income'].transform(lambda x: x.fillna(x.mean()))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


---

**Замена вещественного типа данных на целочисленный**

Анализируя столбцы `days_employed` и `total_income` бросаются в глаза столбцы с отрицательным числами (используем функцию `abs()`для возврата абсолютного значения числа) и цифрами с 6-ю знаками после запятой (используем функцию `astype('int')`). Приведём данные в этих колонках в надлежащий вид.  

In [4]:
data['days_employed'] = abs(data['days_employed'].astype('int'))
data['total_income'] = abs(data['total_income'].astype('int'))
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


---

**Удаление дубликатов**

Выясним есть ли в нашей таблице дубликаты строк методом `duplicated()`, и удалим их методом `drop_duplicates()`.

In [5]:
print('Кол-во дублей = {:.0f}'.format(data.duplicated().sum()))

Кол-во дублей = 54


In [6]:
data.drop_duplicates(inplace = True)
print('Кол-во дублей = {:.0f}'.format(data.duplicated().sum() ) )

Кол-во дублей = 0


В столбце `education` обнаруждено много дублей, связанных с разным написанием одних и тех же слов. Часть вводили строчными, часть заглавными буквами и еще есть слов с загланой первой буквой. Необходимо привести этои слова в нижний регистр методом `str.lower()`. 

In [7]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [8]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [9]:
print('Кол-во дублей = {:.0f}'.format(data.duplicated().sum()))
data['education'].unique()

Кол-во дублей = 17


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

После исправления регистров образовались дубли. Применим повторно метод `drop_duplicates`.

In [10]:
data.drop_duplicates(inplace = True)
print('Кол-во дублей = {:.0f}'.format(data.duplicated().sum()))

Кол-во дублей = 0


In [11]:
data['education_id'].value_counts()

1    15172
0     5250
2      744
3      282
4        6
Name: education_id, dtype: int64

---

В столбце `children`необходимо заменить явно ошибочные данные.

In [12]:
data['children'].value_counts()


 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обращают внимания строки с минусовым количеством детей и цифра "20". 
- Предположу, что это опечатки при заполнении. Соотсветственно, количество "-1" будем считать "1" и добавим к тем, кто имеет одного ребёнка. 
- Очевидно, что цифра "2" трансформировалась в цифру "20" случайно. Возможно, ошибка произошла при заполнении/выгрузке. "20" 
будем считать как "2".

Произведем соответствующую замену и проверим результат.

In [13]:
data.loc[data['children']== -1,'children'] = 1
data.loc[data['children']== 20,'children'] = 2
data['children'].value_counts()


0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64



---

Возраст клиента `dob_years`

Проведём анализ всех возростов кредиторов банка.

In [14]:
print(data['dob_years'].value_counts(ascending=False))

35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
55    443
59    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


В данных о возрасте есть 101 человек с возрастом 0. Выясним сколько из них имели задолженность по возврату кредита.

In [15]:
age_0 = data[data['dob_years'] == 0]
age_0_share = age_0['debt'].count() / data['debt'].count()
print('Доля кредиторов с нулевым возрастом:','{:.1%}'.format(age_0_share))

Доля кредиторов с нулевым возрастом: 0.5%


Всего полпроцента погрешности нашего анализа. Полагаю, что если мы можем не обращать внимание эту "нулевую" группу, это не повлияет на корректность нашего исследования. 

---

**Лемматизация целей кредита** `purpose`

Изучим на что брали кредит - графу `purpose`.

In [16]:
print(data['purpose'].value_counts().count())
print('\n')
print(data['purpose'].value_counts())

38


свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием 

 Из 38 уникальных назначений/цели кредита (разная формулировка), всего можно выделить 4 - свадьба, недвижимость, автомобиль, образование. Думаю, что мы можем объединить жильё и недвижимость, назвав эту цель как "недвижимость" (не принимая во внимание целевого назначения полученных кредита - коммерческая, жилая, строительство, ремонт, т.к. в большинстве случаев, обеспечением возврата такого кредита будет служить собственно сама недвижимость).
 
 Лемматизируем слова в столбце `purpose`, избавимся от предлогов и дополнительных слов ('в',' ','\n','на','супер','для','покупка', 'сдача','семья'), заменим слово 'жильё' на 'недвижимость'.
 
 Добавим лемматизированную колонку в нашу таблицу и посмотрим какие назначений/цели встречаются чаще всего.

In [17]:
from pymystem3 import Mystem
m = Mystem()
string = data['purpose']
oneword_purpose = []
index = 0
stoplist = ['в',' ','\n','на','супер','для','покупка', 'сдача','семья']
text = ''
for row in string:
    index += 1
    for word in m.lemmatize(row):
        
        if word not in stoplist:
            text = word
    oneword_purpose.append(text)
#print(oneword_purpose)


data['aim_all'] = oneword_purpose
data['aim_all'] = data['aim_all'].replace('жилье', 'недвижимость')
data['aim_all'].value_counts()
#data.head()


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: aim_all, dtype: int64

---

**Категоризация данных**

По возрасту клиентов

Чтобы было легче работать с возрастом клиентов, разделим их на группы: "молодой" - до 30 лет, "средний" - с 30 до 60 лет 
и "пожилой" - с 61 года. Добавим столбец `age_group` в нашу таблицу.

In [18]:
def age_groups(years):
        if years < 30:
                return 'молодой'
        if years <= 60:
                return 'средний'
        return 'пожилой'
    
data['age_group'] = data['dob_years'].apply(age_groups)
data.head()
data['age_group'].value_counts()

средний    16047
молодой     3281
пожилой     2126
Name: age_group, dtype: int64

Как можно было предположить, средняя возрастная группа (от 30 до 60 лет) берёт кредиты намного чаще (молодых и пожилых).

---
По уровню дохода

Для определения зависимости между уровнем дохода и возвратом кредита в срок сгруппируем кредиторов на несколько категорий. Согласно исследованию, по [уровню доходов в РФ выделяют](https://dengivsetakipahnyt.com/obrashhenie-s-dengami/sootnoshenie-bogatyx-i-bednyx-v-sovremennoj-rossii.html):

- крайнюю нищету со средним доходом ниже 3 500 рублей;
- нищету это те, кто зарабатывает до 7 500 рублей ежемесячно;
- бедность люди с доходом в 17 тыс. руб.;
- выше бедности - граждане с зарплатой до 25 000 рублей;
- средний достаток, те, кто зарабатывает до 50 тыс. руб. в месяц;
- состоятельные люди получают в месяц до 75 000 рублей;
- богатые, зарабатывающие более 75 000 рублей в месяц.

Изучим данные по доходам клиентов предоставленные нашим банком. Посмотрим минимальные значения и усреднённые.

In [19]:
income_stat = data.groupby('income_type')['total_income']
print(income_stat.min())
print('\n')
print(income_stat.mean().astype('int'))

income_type
безработный         59956
в декрете           53829
госслужащий         29200
компаньон           28702
пенсионер           20667
предприниматель    499163
сотрудник           21367
студент             98201
Name: total_income, dtype: int64


income_type
безработный        131339
в декрете           53829
госслужащий        170897
компаньон          202416
пенсионер          137126
предприниматель    499163
сотрудник          161379
студент             98201
Name: total_income, dtype: int64


Если сопостовить клиентов нашего банка с приведённой выше статистикой, то получается, что минимальный доход пенсионера и сотрудника находится на уровне "выше бедности", а доход студента соответствует категории богатых людей. А при сопоставлении усреднённых значений доходов клиентов банка следует, что все (кроме "в декрете") кредиторы *богачи*.  
Данная категоризация для нашего исследования не подходит.

Предлагаю, воспользовавшись сводной статистикой (функция `describe`), выяснить границы (25%, 50% и 75%) уровня доходов заёмщиков нашего банка. 

In [20]:
data['total_income'].describe().astype('int')

count      21454
mean      167431
std        98060
min        20667
25%       107623
50%       151887
75%       202417
max      2265604
Name: total_income, dtype: int64

Согласно другой [классификации доходов граждан](https://zen.yandex.ru/media/dogecrypto/kak-poschitat-k-kakomu-klassu-vy-otnosites-po-urovniu-dohoda-5b150b5aea0fe700a848e1b1), нижний уровень среднего класса определён в 100 тысяч рублей, предсредний класс - 150000 рублей, средний класс - 250000 рублей, верхний средний класс - 500000, состоятельные - 1млн.руб. Данные уровни уже ближе соотносятся с нашей выборкой, и ими можно воспользоваться для категоризации кредиторов банка.

Приминительно к клиентам данного банка, предлагаю обозначит следующие категории и границы дохода:

- доходы до 60000 - `бедные`(минимальный уровень в нашей выборке для безработного - 59956);
- доходы от 60000 до 107623 - `нижний средний` (25% отсечка в нашей выборке);
- доходы от 107623 до 151887 - `предсредний` (50% отсечка в нашей выборке); 
- доходы от 151887 до 202417 - `средний` (75% отсечка в нашей выборке);
- доходы от 202417 до 500000 - `состоятельные`;
- доходы свыше 500000 - `богатые`.

In [21]:
def income_pc(total):
        if total < 60000:
                return 'бедные'
        if total <= 107623:
                return 'нижний средний'
        if total <= 151887:
                return 'предсредний'
        if total <= 202417:
                return 'средний'
        if total <= 500000:
                return 'состоятельные'    
        return 'богатые'
    
data['income_group'] = data['total_income'].apply(income_pc)
print(data['income_group'].value_counts())

средний           5818
предсредний       5363
состоятельные     4687
нижний средний    4558
бедные             806
богатые            222
Name: income_group, dtype: int64


---
По семейному положению 

В таблице 2 колонки, представляющие нам информацию о семейном положении - `family_status` и `family_status_id`, причём, последняя (`id`) представляет для нашего исследования больший интерес.

In [22]:
print(data['family_status'].value_counts())
data['family_status_id'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

Для банка важен статус заёмщика де-юре (женат/замужем или холост/не замужняя), остальное - лирика (гражданский брак, разведён, вдовец). Поэтому выделим два статуса семейного положения  - женат/замужем (0) и одиночка (1). Соответственно, `family_status_id` с 1 по 4 примем за 1 (одиночка).

In [23]:
for id in data['family_status_id']:
    if id > 1:
        data.loc[data['family_status_id']== id,'family_status_id'] = 1

data['family_status_id'].value_counts()

0    12339
1     9115
Name: family_status_id, dtype: int64

Итак, статус кредиторов в нашей таблице:
- 12339 - женат/замужем;
- 9115 - холост/не замужняя.
---

### 3. Ответы на вопросы относительно факторов влияющих на погашения кредита в срок. 


**Задолженность по возврату кредитов**  `debt`  

In [24]:
print(data['debt'].value_counts())
debtors = data.loc[data['debt']== 1] #недобросовестные заёмщики

0    19713
1     1741
Name: debt, dtype: int64


В нашей выборке 1741 человек не погасивший свой кредит в срок.

**Половая принадлежность**  `gender`

In [25]:
data['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

Думаю, 1 человек неизвестного пола не испортит наш анализ.

---

### Расчет коэффициентов риска

Итак, я привел в порядок полученные данные и теперь можно приступить к ответу на вопрос заказчика - *влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок*. Замечу, что в задании также указано, что данные этого исследования будут использованы при построении модели кредитного скоринга. Данные для скоринговых систем получаются из вероятностей/рисков невозвратов кредитов отдельными группами заёмщиков, полученными из анализа кредитной истории тысяч людей.
Так как в нашем случае у нас есть информация о факте существования задолженности/невозврата, можно определить корреляцию между определенными социальными данными (наличие детей, отношение к браку, наличие высшего образования) и добросовестностью заемщика.

Принимая во внимание поставленную задачу, сначала ответим на два основных вопроса.

Для начала выделим в `aim_debtors` тех, кто не вернул кредит в срок.


In [26]:
aim_debtors = data[data['debt'] == 1]
aim_debtors['aim_all'].value_counts()

недвижимость    782
автомобиль      403
образование     370
свадьба         186
Name: aim_all, dtype: int64

Исследуем как на своевременный возврат кредита влияет семейное положение и количество детей. Выясним взаимосвязь между показателем погашения кредита в срок и двумя другими параметрами - семейным положением и количеством детей. Воспользуемся для этого сводной таблицей, вызвав метод `pivot_table()`.

In [27]:
print(data['children'].value_counts())
print(aim_debtors['children'].value_counts())

print('\n')

data_pivot = data.pivot_table('debt', index=['family_status_id'], columns='children')
data_pivot

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
0    1063
1     445
2     202
3      27
4       4
Name: children, dtype: int64




children,0,1,2,3,4,5
family_status_id,,,,,,
0,0.069095,0.082224,0.093552,0.068273,0.103448,0.0
1,0.082591,0.106969,0.098901,0.123457,0.083333,0.0


> *Я бы не стал учитывать в данном исследовании кредиторов с 3 (330/27) и более детьми (50/4), т.к. эта группа заёмщиков немногочисленна и не может считаться репрезентативной.*

---
Из этой сводной таблицы видно, что **чаще всего не возвращают кредиты в срок одиночки с одним и двумя детьми**. Бездетные как семейные, так и одиночки, наиболее дисциплинированы. **Реже всего просрачивают сроки бездетные женатые/замужние**.   

In [28]:
ratio_couple = (aim_debtors[aim_debtors['family_status_id'] == 0]['family_status_id'].count()) / (data[data['family_status_id'] == 0]['family_status_id'].count())
print('Соотношение просроченных кредитов взятых семейными:','{:.2}'.format(ratio_couple))
print('\n')
ratio_single = (aim_debtors[aim_debtors['family_status_id'] == 1]['family_status_id'].count()) / (data[data['family_status_id'] == 1]['family_status_id'].count())
print('Соотношение просроченных кредитов взятых одиночками:','{:.2}'.format(ratio_single))


Соотношение просроченных кредитов взятых семейными: 0.075


Соотношение просроченных кредитов взятых одиночками: 0.089


In [29]:
ratio_child_0 = (aim_debtors[aim_debtors['children'] == 0]['children'].count()) / (data[data['children'] == 0]['children'].count())
print('Соотношение просроченных кредитов взятых бездетными:','{:.2}'.format(ratio_child_0))
print('\n')
ratio_child_1 = (aim_debtors[aim_debtors['children'] == 1]['children'].count()) / (data[data['children'] == 1]['children'].count())
print('Соотношение просроченных кредитов взятых родителем с 1 ребёнком:','{:.2}'.format(ratio_child_1))
print('\n')
ratio_child_2 = (aim_debtors[aim_debtors['children'] == 2]['children'].count()) / (data[data['children'] == 2]['children'].count())
print('Соотношение просроченных кредитов взятых родителем с 2 детьми:','{:.2}'.format(ratio_child_2))
print('\n')
ratio_child_3 = (aim_debtors[aim_debtors['children'] == 3]['children'].count()) / (data[data['children'] == 3]['children'].count())
print('Соотношение просроченных кредитов взятых родителем с 3 детьми:','{:.2}'.format(ratio_child_3))
print('\n')
ratio_child_4 = (aim_debtors[aim_debtors['children'] == 4]['children'].count()) / (data[data['children'] == 4]['children'].count())
print('Соотношение просроченных кредитов взятых родителем с 4 детьми:','{:.2}'.format(ratio_child_4))

Соотношение просроченных кредитов взятых бездетными: 0.075


Соотношение просроченных кредитов взятых родителем с 1 ребёнком: 0.092


Соотношение просроченных кредитов взятых родителем с 2 детьми: 0.095


Соотношение просроченных кредитов взятых родителем с 3 детьми: 0.082


Соотношение просроченных кредитов взятых родителем с 4 детьми: 0.098


---

Чаще всего по количественному показателю обращаются за кредитом связанным с недвижимостью и, как видим, чаще не возвращают в срок.   

In [30]:
ratio_est = (aim_debtors[aim_debtors['aim_all'] == 'недвижимость']['aim_all'].count()) / (data[data['aim_all'] == 'недвижимость']['aim_all'].count())
print('Соотношение просроченных кредитов взятых на недвижимость:','{:.2}'.format(ratio_est))
print('\n')
ratio_auto = (aim_debtors[aim_debtors['aim_all'] == 'автомобиль']['aim_all'].count()) / (data[data['aim_all'] == 'автомобиль']['aim_all'].count())
print('Соотношение просроченных кредитов взятых на автомобиль:','{:.2}'.format(ratio_auto))
print('\n')
ratio_educ = (aim_debtors[aim_debtors['aim_all'] == 'образование']['aim_all'].count()) / (data[data['aim_all'] == 'образование']['aim_all'].count())
print('Соотношение просроченных кредитов взятых на образование:','{:.2}'.format(ratio_educ))
print('\n')
ratio_wedd = (aim_debtors[aim_debtors['aim_all'] == 'свадьба']['aim_all'].count()) / (data[data['aim_all'] == 'свадьба']['aim_all'].count())
print('Соотношение просроченных кредитов взятых на свадьбу:','{:.2}'.format(ratio_wedd))

Соотношение просроченных кредитов взятых на недвижимость: 0.072


Соотношение просроченных кредитов взятых на автомобиль: 0.094


Соотношение просроченных кредитов взятых на образование: 0.092


Соотношение просроченных кредитов взятых на свадьбу: 0.08


Однако соотношения показывают нам, что чаще не возвращают своевременно займ, взятый на покупку автомобиля и на оплату образования. Реже всего не соблюдают сроки по кредитам, связанным с недвижимостью. 

---

Как возраст влияет на своевременность возврата кредита?

In [31]:
ratio_age_young = (aim_debtors[aim_debtors['age_group'] == 'молодой']['age_group'].count()) / (data[data['age_group'] == 'молодой']['age_group'].count())
print('Соотношение просроченных кредитов взятых молодыми (до 30 лет):','{:.2}'.format(ratio_age_young))
print('\n')
ratio_age_middle = (aim_debtors[aim_debtors['age_group'] == 'средний']['age_group'].count()) / (data[data['age_group'] == 'средний']['age_group'].count())
print('Соотношение просроченных кредитов взятых лицами среднего возраста (от 30 до 60 лет):','{:.2}'.format(ratio_age_middle))
print('\n')
ratio_age_senior = (aim_debtors[aim_debtors['age_group'] == 'пожилой']['age_group'].count()) / (data[data['age_group'] == 'пожилой']['age_group'].count())
print('Соотношение просроченных кредитов взятых пожилыми (старше 60 лет):','{:.2}'.format(ratio_age_senior))


Соотношение просроченных кредитов взятых молодыми (до 30 лет): 0.11


Соотношение просроченных кредитов взятых лицами среднего возраста (от 30 до 60 лет): 0.08


Соотношение просроченных кредитов взятых пожилыми (старше 60 лет): 0.048


---

Интересно проанализировать влияет ли гендерная принадлежность на факт погашения кредита в срок.

In [32]:
print(aim_debtors['gender'].value_counts())
print(data['gender'].value_counts())
print('\n')
ratio_female = (aim_debtors[aim_debtors['gender'] == 'F']['gender'].count()) / (data[data['gender'] == 'F']['gender'].count())
print('Соотношение просроченных кредитов взятых женщинами:','{:.2}'.format(ratio_female))
ratio_male = (aim_debtors[aim_debtors['gender'] == 'M']['gender'].count()) / (data[data['gender'] == 'M']['gender'].count())
print('Соотношение просроченных кредитов взятых мужчинами:','{:.2}'.format(ratio_male))

F    994
M    747
Name: gender, dtype: int64
F      14174
M       7279
XNA        1
Name: gender, dtype: int64


Соотношение просроченных кредитов взятых женщинами: 0.07
Соотношение просроченных кредитов взятых мужчинами: 0.1


Хотя женщины почти в 2 раза чаще берут кредиты, не возвращают кредит в срок они реже. Мужчины чаще нарушают сроки возврата.

---

Выясним взаимосвязь между показателем возврата кредита в срок, категорией дохода кредитора и целью кредита при помощи сводной таблицы `pivot_table`. Причем, с помощью параметра margins вычислим обобщенные значения по каждой группе.

In [33]:
data.pivot_table('debt', index = 'income_group', columns = 'aim_all', margins=True).sort_values('All',ascending=False)

aim_all,автомобиль,недвижимость,образование,свадьба,All
income_group,,,,,
предсредний,0.097797,0.079245,0.100798,0.092014,0.088570
средний,0.096688,0.075402,0.098450,0.085484,0.084909
нижний средний,0.095455,0.069850,0.095721,0.096591,0.082931
All,0.093590,0.072334,0.092200,0.080034,0.081150
состоятельные,0.082474,0.067397,0.078759,0.049505,0.070621
богатые,0.075000,0.047244,0.117647,0.047619,0.063063
бедные,0.103659,0.055556,0.038961,0.040541,0.060794


Чаще всего несвоевременно возвращают кредит заёмщики с доходом "предсредний" (от 107623 до 151887) - 8,9%, "средний"(151887 до 202417) - 8,5% и "нижний средний"(60000 до 107623) - 8,3%. Эти группы нарушают сроки чаще среднего значения (8,1%).
Чаще придерживаются сроков возврата кредита категории "бедные"(до 60000) - 6,1% и "богатые" (свыше 500000), но это и самые малочисленные категории заёмщиков банка.

Относительно цели кредита, чаще всего нарушают сроки заёмщики из категории "богатые" (свыше 500000) на образование - 11,8%, "бедные"(до 60000) на покупку автомобиля - 10% и "предсредний" (от 107623 до 151887) на образование - 10%. Чаще возвращают кредит в срок клиенты из группы "бедные"(до 60000) на свадьбу ~ 4,1% и "богатые" (свыше 500000) на покупку недвижимости и на свадьбу - 4,7%. 

---

А как образование влияет на своевременное погашение кредитов?

In [34]:
ratio_educ_0 = (aim_debtors[aim_debtors['education_id'] == 0]['education_id'].count()) / (data[data['education_id'] == 0]['education_id'].count())
print('Соотношение просроченных кредитов взятых лицами со среднем образованием:','{:.2}'.format(ratio_educ_0))
print('\n')
ratio_educ_1 = (aim_debtors[aim_debtors['education_id'] == 1]['education_id'].count()) / (data[data['education_id'] == 1]['education_id'].count())
print('Соотношение просроченных кредитов взятых лицами с высшим образованием:','{:.2}'.format(ratio_educ_1))
print('\n')
ratio_educ_2 = (aim_debtors[aim_debtors['education_id'] == 2]['education_id'].count()) / (data[data['education_id'] == 2]['education_id'].count())
print('Соотношение просроченных кредитов взятых лицами с неоконченным высшим:','{:.2}'.format(ratio_educ_2))
print('\n')
ratio_educ_3 = (aim_debtors[aim_debtors['education_id'] == 3]['education_id'].count()) / (data[data['education_id'] == 3]['education_id'].count())
print('Соотношение просроченных кредитов взятых лицами с начальным образованием:','{:.2}'.format(ratio_educ_3))

Соотношение просроченных кредитов взятых лицами со среднем образованием: 0.053


Соотношение просроченных кредитов взятых лицами с высшим образованием: 0.09


Соотношение просроченных кредитов взятых лицами с неоконченным высшим: 0.091


Соотношение просроченных кредитов взятых лицами с начальным образованием: 0.11


Я бы выделил риск несвоевременного возврата кредита у людей с начальным и неоконченным высшим.

---
> Так как моё исследование будет учтено при построении модели кредитного скоринга, для удобства анализа оценки способности потенциального заёмщика вернуть кредит банку, соберу все коэффициенты в одну таблицу и cгруппирую коэффициенты по степени риска (высокий, средний, низкий).

In [35]:
column_opt = ['параметр', 'коэффициент']
rat = (['на недвижимость',ratio_est], ['на автомобиль', ratio_auto], ['на образование', ratio_educ], ['на свадьбу', ratio_wedd], 
       ['женат/замужем', ratio_couple], ['одиночки', ratio_single], ['бездетные', ratio_child_0], ['1 ребёнок',ratio_child_1], 
       ['2 ребёнка', ratio_child_2], ['3 ребёнка', ratio_child_3], ['4 ребёнка', ratio_child_4], 
       ['категория до 30 лет', ratio_age_young], ['категория от 30 до 60 лет', ratio_age_middle], 
       ['от 60 и старше', ratio_age_senior], ['среднее образование', ratio_educ_0], ['высшее образование', ratio_educ_1], 
       ['неоконченное высшее', ratio_educ_2], ['начальное образование', ratio_educ_3], ['мужчины', ratio_male], 
       ['женщины', ratio_female])
rat_o = pd.DataFrame(rat, columns = column_opt)
rating_sorted = rat_o.sort_values(by = 'коэффициент', ascending = False)
rating_sorted

,параметр,коэффициент
17,начальное образование,0.109929
11,категория до 30 лет,0.108808
18,мужчины,0.102624
10,4 ребёнка,0.097561
8,2 ребёнка,0.094925
1,на автомобиль,0.093590
2,на образование,0.092200
7,1 ребёнок,0.091658
16,неоконченное высшее,0.091398
15,высшее образование,0.089902


**Для того, чтобы заказчику легче было ориентироваться в моих выводах, определю *условную* степень риска для банка, чтобы легче было учитывать моё исследование в оценке способности заёмщика вернуть кредит банку.** 

In [36]:
def alert_rating(rate):
    #rating = rate('коэффициент')
    if rate <= 0.079951:
        return 'низкий'
    if rate < 0.09:
        return 'средний'
    return 'высокий'

rating_sorted['risk'] = rating_sorted['коэффициент'].apply(alert_rating)
rating_sorted

,параметр,коэффициент,risk
17,начальное образование,0.109929,высокий
11,категория до 30 лет,0.108808,высокий
18,мужчины,0.102624,высокий
10,4 ребёнка,0.097561,высокий
8,2 ребёнка,0.094925,высокий
1,на автомобиль,0.093590,высокий
2,на образование,0.092200,высокий
7,1 ребёнок,0.091658,высокий
16,неоконченное высшее,0.091398,высокий
15,высшее образование,0.089902,средний


### Выводы:
1. Семейное положение влияет на факт погашения кредита в срок. Хотя, количество недобросовестных женатых/замужних в нашей выборке больше, но соотношение просроченных к количеству взятых кредитов говорит нам о том, что **одиночки чаще нарушают сроки и несвоевременно возвращают деньги**.
2. Количество детей также влияет на факт погашения кредита в срок. **Бездетные женатые/замужние реже нарушают сроки возврата кредита**. Чаще всего просрачивают сроки заёмщики с 2 и одним ребёнком. Многодетные (3-4 ребёнка) реже нарушают сроки, хотя этот вывод не репрезентативен из-за малой статистики.

#### Дополнительные выводы:

- Чем моложе заёмщик, тем выше риск невозврата кредита(от 30 до 60 лет). Чаще всего не соблюдают сроки кредиторы до 30 лет.
- Чаще не возвращают в срок кредиты, связанные с покупкой автомобиля и оплатой образования. Риск по этим целевым категориям выше, нежели по кредитам связанным с недвижимостью и свадьбой. 
- Женщины почти в 2 раза чаще берут кредиты и дисциплинированнее мужчин относятся к своим обязательствам (возвращают кредит в срок). Мужчины чаще нарушают сроки возврата.
- Чаще всего несвоевременно возвращают кредит заёмщики с ежемесячным доходом от 107623 до 202417.
- Чаще всего нарушают сроки возврата богатые клиенты с доходом свыше 500000 тысяч в месяц, получившие кредит на оплату образования; заёмщики с доходом до 60000 рублей на покупку автомобиля и займщики с доходом от 107623 до 151887 рублей на опплату образования.
- Своевременно возвращают кредит клиенты из группы с доходом до 60000 на свадьбу и "богатые" с доходом свыше 500000 на покупку недвижимости и на свадьбу.

#### Рекомендация:

- Учитывать обозначенную в данном исследовании степень риска при построении модели кредитного скоринга.